In [12]:
## クリーニング処理
def cleaning_reviews(reviews):
    dust_keywords = ['<br />', '【', '】', '(', ')','（',  '）','「', '」', '＋', '+','★', '☆', '※','~', '／', '〜', '*', '^', 'm(_ _)m', '(^^)', '(^^)', 'm(._.)m', 'ヽ=ﾟДﾟ=ﾉ']
    for key in dust_keywords:
        reviews['コメント'] = reviews['コメント'].str.replace(key, '')
    return reviews

In [26]:
## 単語の分割
import json
from sudachipy import tokenizer
from sudachipy import dictionary
from sudachipy import config

def words_tokenize(reviews):
    with open(config.SETTINGFILE, "r", encoding="utf-8") as f:
        settings = json.load(f)
    tokenizer_obj = dictionary.Dictionary(settings).create()
    mode = tokenizer.Tokenizer.SplitMode.C
    for i, comment in enumerate(reviews['コメント']):
        for m in tokenizer_obj.tokenize(mode, comment):
            print(m.surface())
            print(m.part_of_speech())
#       [m.surface() for m in tokenizer_obj.tokenize(mode, comment)]

In [27]:
## メイン関数
import sys
import codecs
import random
import numpy as np
import numpy.random
import pandas as pd
import copy
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Conv1D, MaxPooling1D, Dropout, LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD
from keras.utils import np_utils

## ファイルの読込み(全件)
#shops = pd.read_csv('shops.txt', delimiter='\t')
reviews = pd.read_csv('reviews.txt', delimiter='\t')

## クリーニング
reviews = cleaning_reviews(reviews)

## 単語分割
words_tokenize(reviews)

print(reviews['コメント'])

三鷹駅
['名詞', '固有名詞', '地名', '一般', '*', '*']
から
['助詞', '格助詞', '*', '*', '*', '*']
結構
['副詞', '*', '*', '*', '*', '*']
歩き
['動詞', '一般', '*', '*', '五段-カ行', '連用形-一般']
ます
['助動詞', '*', '*', '*', '助動詞-マス', '終止形-一般']
。
['補助記号', '句点', '*', '*', '*', '*']
曜日
['名詞', '普通名詞', '一般', '*', '*', '*']
に
['助詞', '格助詞', '*', '*', '*', '*']
よっ
['動詞', '一般', '*', '*', '五段-ラ行', '連用形-促音便']
て
['助詞', '接続助詞', '*', '*', '*', '*']
味
['名詞', '普通名詞', '一般', '*', '*', '*']
が
['助詞', '格助詞', '*', '*', '*', '*']
違う
['動詞', '一般', '*', '*', '五段-ワア行', '連体形-一般']
の
['助詞', '準体助詞', '*', '*', '*', '*']
と
['助詞', '格助詞', '*', '*', '*', '*']
、
['補助記号', '読点', '*', '*', '*', '*']
夜
['名詞', '普通名詞', '副詞可能', '*', '*', '*']
の
['助詞', '格助詞', '*', '*', '*', '*']
営業
['名詞', '普通名詞', 'サ変可能', '*', '*', '*']
が
['助詞', '格助詞', '*', '*', '*', '*']
ない
['形容詞', '非自立可能', '*', '*', '形容詞', '連体形-一般']
の
['助詞', '準体助詞', '*', '*', '*', '*']
で
['助動詞', '*', '*', '*', '助動詞-ダ', '連用形-一般']
休日
['名詞', '普通名詞', '副詞可能', '*', '*', '*']
に
['助詞', '格助詞', '*', '*', '*', '*']
あたる
['動詞', '一

['助詞', '格助詞', '*', '*', '*', '*']
し
['動詞', '非自立可能', '*', '*', 'サ行変格', '連用形-一般']
た
['助動詞', '*', '*', '*', '助動詞-タ', '連体形-一般']
ピリ
['名詞', '固有名詞', '人名', '一般', '*', '*']
感
['名詞', '普通名詞', '一般', '*', '*', '*']
を
['助詞', '格助詞', '*', '*', '*', '*']
感じる
['動詞', '一般', '*', '*', '上一段-ザ行', '連体形-一般']
うま
['形容詞', '一般', '*', '*', '形容詞', '語幹-一般']
さ
['接尾辞', '名詞的', '一般', '*', '*', '*']
です
['助動詞', '*', '*', '*', '助動詞-デス', '終止形-一般']
。
['補助記号', '句点', '*', '*', '*', '*']
具材
['名詞', '普通名詞', '一般', '*', '*', '*']
豊富
['形状詞', '一般', '*', '*', '*', '*']
な
['助動詞', '*', '*', '*', '助動詞-ダ', '連体形-一般']
濃口
['名詞', '普通名詞', '一般', '*', '*', '*']
ミソ
['名詞', '普通名詞', '一般', '*', '*', '*']
でし
['助動詞', '*', '*', '*', '助動詞-デス', '連用形-一般']
た
['助動詞', '*', '*', '*', '助動詞-タ', '終止形-一般']
。
['補助記号', '句点', '*', '*', '*', '*']
スープ
['名詞', '普通名詞', '一般', '*', '*', '*']
が
['助詞', '格助詞', '*', '*', '*', '*']
具材
['名詞', '普通名詞', '一般', '*', '*', '*']
に
['助詞', '格助詞', '*', '*', '*', '*']
染み込む
['動詞', '一般', '*', '*', '五段-マ行', '連体形-一般']
事
['名詞', '普通名詞', '一般', '*', 

暫く
['副詞', '*', '*', '*', '*', '*']
お
['接頭辞', '*', '*', '*', '*', '*']
休み
['動詞', '一般', '*', '*', '五段-マ行', '連用形-一般']
し
['動詞', '非自立可能', '*', '*', 'サ行変格', '連用形-一般']
て
['助詞', '接続助詞', '*', '*', '*', '*']
おり
['動詞', '非自立可能', '*', '*', '五段-ラ行', '連用形-一般']
まし
['助動詞', '*', '*', '*', '助動詞-マス', '連用形-一般']
た
['助動詞', '*', '*', '*', '助動詞-タ', '連体形-一般']
の
['助詞', '準体助詞', '*', '*', '*', '*']
で
['助動詞', '*', '*', '*', '助動詞-ダ', '連用形-一般']
過去
['名詞', '普通名詞', '副詞可能', '*', '*', '*']
の
['助詞', '格助詞', '*', '*', '*', '*']
記録
['名詞', '普通名詞', 'サ変可能', '*', '*', '*']
と
['助詞', '格助詞', '*', '*', '*', '*']
し
['動詞', '非自立可能', '*', '*', 'サ行変格', '連用形-一般']
て
['助詞', '接続助詞', '*', '*', '*', '*']
レビュー
['名詞', '普通名詞', 'サ変可能', '*', '*', '*']
up
['名詞', '普通名詞', 'サ変可能', '*', '*', '*']
です
['助動詞', '*', '*', '*', '助動詞-デス', '終止形-一般']
。
['補助記号', '句点', '*', '*', '*', '*']
九州
['名詞', '固有名詞', '地名', '一般', '*', '*']
・
['補助記号', '一般', '*', '*', '*', '*']
久留米
['名詞', '固有名詞', '地名', '一般', '*', '*']
の
['助詞', '格助詞', '*', '*', '*', '*']
名店
['名詞', '普通名詞', '一般', '

ストレート
['名詞', '普通名詞', '形状詞可能', '*', '*', '*']
に
['助詞', '格助詞', '*', '*', '*', '*']
伝わる
['動詞', '一般', '*', '*', '五段-ラ行', '連体形-一般']
こちら
['代名詞', '*', '*', '*', '*', '*']
の
['助詞', '格助詞', '*', '*', '*', '*']
和風
['名詞', '普通名詞', '一般', '*', '*', '*']
カエシ
['名詞', '普通名詞', '一般', '*', '*', '*']
は
['助詞', '係助詞', '*', '*', '*', '*']
かなり
['形状詞', '一般', '*', '*', '*', '*']
の
['助詞', '格助詞', '*', '*', '*', '*']
拘り
['動詞', '一般', '*', '*', '文語四段-ラ行', '連用形-一般']
と
['助詞', '接続助詞', '*', '*', '*', '*']
コスト
['名詞', '普通名詞', '一般', '*', '*', '*']
を
['助詞', '格助詞', '*', '*', '*', '*']
かけ
['動詞', '非自立可能', '*', '*', '下一段-カ行', '連用形-一般']
て
['助詞', '接続助詞', '*', '*', '*', '*']
いる
['動詞', '非自立可能', '*', '*', '上一段-ア行', '終止形-一般']
と
['助詞', '格助詞', '*', '*', '*', '*']
聴い
['動詞', '一般', '*', '*', '五段-カ行', '連用形-イ音便']
て
['助詞', '接続助詞', '*', '*', '*', '*']
おり
['動詞', '非自立可能', '*', '*', '五段-ラ行', '連用形-一般']
ます
['助動詞', '*', '*', '*', '助動詞-マス', '終止形-一般']
。
['補助記号', '句点', '*', '*', '*', '*']
これ
['代名詞', '*', '*', '*', '*', '*']
に
['助詞', '格助詞', '*', '*', '*',

コチラ
['代名詞', '*', '*', '*', '*', '*']
も
['助詞', '係助詞', '*', '*', '*', '*']
RDB
['名詞', '普通名詞', '一般', '*', '*', '*']
おやすみ
['感動詞', '一般', '*', '*', '*', '*']
期間
['名詞', '普通名詞', '一般', '*', '*', '*']
中
['接尾辞', '名詞的', '副詞可能', '*', '*', '*']
レビュー
['名詞', '普通名詞', 'サ変可能', '*', '*', '*']
です
['助動詞', '*', '*', '*', '助動詞-デス', '終止形-一般']
。
['補助記号', '句点', '*', '*', '*', '*']
丁度
['副詞', '*', '*', '*', '*', '*']
お
['接頭辞', '*', '*', '*', '*', '*']
やすみ
['名詞', '普通名詞', '一般', '*', '*', '*']
に
['助詞', '格助詞', '*', '*', '*', '*']
入る
['動詞', '一般', '*', '*', '五段-ラ行', '連体形-一般']
直前
['名詞', '普通名詞', '副詞可能', '*', '*', '*']
の
['助詞', '格助詞', '*', '*', '*', '*']
２
['名詞', '数詞', '*', '*', '*', '*']
年
['名詞', '普通名詞', '助数詞可能', '*', '*', '*']
前
['名詞', '普通名詞', '副詞可能', '*', '*', '*']
の
['助詞', '格助詞', '*', '*', '*', '*']
下書き
['名詞', '普通名詞', 'サ変可能', '*', '*', '*']
から
['助詞', '格助詞', '*', '*', '*', '*']
。
['補助記号', '句点', '*', '*', '*', '*']
RDB
['名詞', '普通名詞', '一般', '*', '*', '*']
麺
['名詞', '普通名詞', '一般', '*', '*', '*']
友
['名詞', '普通名詞', '一般', '*', '

...

も
['助詞', '係助詞', '*', '*', '*', '*']
良い
['形容詞', '非自立可能', '*', '*', '形容詞', '終止形-一般']
し
['助詞', '接続助詞', '*', '*', '*', '*']
シッカリ
['副詞', '*', '*', '*', '*', '*']
と
['助詞', '格助詞', '*', '*', '*', '*']
混ぜる
['動詞', '一般', '*', '*', '下一段-ザ行', '終止形-一般']
の
['助詞', '準体助詞', '*', '*', '*', '*']
で
['助動詞', '*', '*', '*', '助動詞-ダ', '連用形-一般']
も
['助詞', '係助詞', '*', '*', '*', '*']
！
['補助記号', '句点', '*', '*', '*', '*']
混ぜ
['動詞', '一般', '*', '*', '下一段-ザ行', '連用形-一般']
加減
['名詞', '普通名詞', 'サ変可能', '*', '*', '*']
は
['助詞', '係助詞', '*', '*', '*', '*']
お
['接頭辞', '*', '*', '*', '*', '*']
好み
['名詞', '普通名詞', '一般', '*', '*', '*']
で
['助詞', '格助詞', '*', '*', '*', '*']
良い
['形容詞', '非自立可能', '*', '*', '形容詞', '終止形-一般']
と
['助詞', '格助詞', '*', '*', '*', '*']
思い
['動詞', '一般', '*', '*', '五段-ワア行', '連用形-一般']
ます
['助動詞', '*', '*', '*', '助動詞-マス', '終止形-一般']
よ
['助詞', '終助詞', '*', '*', '*', '*']
具材
['名詞', '普通名詞', '一般', '*', '*', '*']
＆
['補助記号', '一般', '*', '*', '*', '*']
麺
['名詞', '普通名詞', '一般', '*', '*', '*']
＆
['補助記号', '一般', '*', '*', '*', '*']
タレ
['名詞', '

まわり
['名詞', '普通名詞', '助数詞可能', '*', '*', '*']
目
['接尾辞', '名詞的', '一般', '*', '*', '*']
で
['助詞', '格助詞', '*', '*', '*', '*']
席
['名詞', '普通名詞', '一般', '*', '*', '*']
に
['助詞', '格助詞', '*', '*', '*', '*']
着く
['動詞', '一般', '*', '*', '五段-カ行', '連体形-一般']
こと
['名詞', '普通名詞', '一般', '*', '*', '*']
が
['助詞', '格助詞', '*', '*', '*', '*']
でき
['動詞', '非自立可能', '*', '*', '上一段-カ行', '連用形-一般']
た
['助動詞', '*', '*', '*', '助動詞-タ', '終止形-一般']
。
['補助記号', '句点', '*', '*', '*', '*']
メニュー
['名詞', '普通名詞', '一般', '*', '*', '*']
は
['助詞', '係助詞', '*', '*', '*', '*']
二
['名詞', '数詞', '*', '*', '*', '*']
択
['接尾辞', '名詞的', '一般', '*', '*', '*']
。
['補助記号', '句点', '*', '*', '*', '*']
炊き込みご飯
['名詞', '普通名詞', '一般', '*', '*', '*']
を
['助詞', '格助詞', '*', '*', '*', '*']
つける
['動詞', '非自立可能', '*', '*', '下一段-カ行', '終止形-一般']
か
['助詞', '副助詞', '*', '*', '*', '*']
つけ
['動詞', '非自立可能', '*', '*', '下一段-カ行', '未然形-一般']
ない
['助動詞', '*', '*', '*', '助動詞-ナイ', '終止形-一般']
か
['助詞', '終助詞', '*', '*', '*', '*']
。
['補助記号', '句点', '*', '*', '*', '*']
今回
['名詞', '普通名詞', '副詞可能', '*', '*', '*'

['名詞', '普通名詞', '助数詞可能', '*', '*', '*']
位
['接尾辞', '名詞的', '一般', '*', '*', '*']
で
['助詞', '格助詞', '*', '*', '*', '*']
腹
['名詞', '普通名詞', '一般', '*', '*', '*']
８
['名詞', '数詞', '*', '*', '*', '*']
分
['接尾辞', '名詞的', 'サ変可能', '*', '*', '*']
目
['接尾辞', '名詞的', '一般', '*', '*', '*']
と
['助詞', '格助詞', '*', '*', '*', '*']
いっ
['動詞', '一般', '*', '*', '五段-ワア行', '連用形-促音便']
た
['助動詞', '*', '*', '*', '助動詞-タ', '連体形-一般']
ところ
['名詞', '普通名詞', '副詞可能', '*', '*', '*']
で
['助詞', '格助詞', '*', '*', '*', '*']
、
['補助記号', '読点', '*', '*', '*', '*']
中盛
['名詞', '固有名詞', '人名', '姓', '*', '*']
３００
['名詞', '数詞', '*', '*', '*', '*']
ｇ
['名詞', '普通名詞', '助数詞可能', '*', '*', '*']
迄
['助詞', '副助詞', '*', '*', '*', '*']
同一
['形状詞', '一般', '*', '*', '*', '*']
料金
['名詞', '普通名詞', '一般', '*', '*', '*']
です
['助動詞', '*', '*', '*', '助動詞-デス', '終止形-一般']
。
['補助記号', '句点', '*', '*', '*', '*']
つけ汁
['名詞', '普通名詞', '一般', '*', '*', '*']
は
['助詞', '係助詞', '*', '*', '*', '*']
酸味
['名詞', '普通名詞', '一般', '*', '*', '*']
と
['助詞', '格助詞', '*', '*', '*', '*']
辛味
['名詞', '普通名詞', '一般', '*', '*

['名詞', '普通名詞', '一般', '*', '*', '*']
を
['助詞', '格助詞', '*', '*', '*', '*']
合わせる
['動詞', '一般', '*', '*', '下一段-サ行', '終止形-一般']
と
['助詞', '接続助詞', '*', '*', '*', '*']
、
['補助記号', '読点', '*', '*', '*', '*']
ほどよく
['形容詞', '一般', '*', '*', '形容詞', '連用形-一般']
水菜
['名詞', '普通名詞', '一般', '*', '*', '*']
や
['助詞', '副助詞', '*', '*', '*', '*']
玉ねぎ
['名詞', '普通名詞', '一般', '*', '*', '*']
も
['助詞', '係助詞', '*', '*', '*', '*']
絡ん
['動詞', '一般', '*', '*', '五段-マ行', '連用形-撥音便']
で
['助詞', '接続助詞', '*', '*', '*', '*']
くる
['動詞', '非自立可能', '*', '*', 'カ行変格', '連体形-一般']
の
['助詞', '準体助詞', '*', '*', '*', '*']
で
['助動詞', '*', '*', '*', '助動詞-ダ', '連用形-一般']
、
['補助記号', '読点', '*', '*', '*', '*']
色々
['形状詞', '一般', '*', '*', '*', '*']
な
['助動詞', '*', '*', '*', '助動詞-ダ', '連体形-一般']
味
['名詞', '普通名詞', '一般', '*', '*', '*']
が
['助詞', '格助詞', '*', '*', '*', '*']
一気
['名詞', '普通名詞', '一般', '*', '*', '*']
に
['助詞', '格助詞', '*', '*', '*', '*']
楽しめ
['動詞', '一般', '*', '*', '下一段-マ行', '連用形-一般']
ます
['助動詞', '*', '*', '*', '助動詞-マス', '終止形-一般']
。
['補助記号', '句点', '*', '*', '*', '*']
そ

KeyboardInterrupt: 